# Кибериммунная автономность$\\$Создание конструктивно защищённого автономного наземного транспортного средства$\\$Модуль 2

## О документе

Версия 1.03

Модуль 2 для регионального этапа соревнований по кибериммунной автономности

## Подготовка к работе c модулем 2

Перенесите в блок ниже ваши реализации классов коммуникационного шлюза (CommunicationGateway), систему навигации (NavigationSystem), управления (ControlSystem)

In [9]:
# ваш код
from src.control_system import BaseControlSystem
from src.navigation_system import BaseNavigationSystem
from src.communication_gateway import BaseCommunicationGateway
from multiprocessing import Queue
from src.config import SERVOS_QUEUE_NAME
from src.config import CONTROL_SYSTEM_QUEUE_NAME
from src.config import CARGO_BAY_QUEUE_NAME
from src.event_types import Event


class ControlSystem(BaseControlSystem):
    """ControlSystem блок расчёта управления """

    def _send_speed_and_direction_to_consumers(self, speed, direction):
        servos_q_name = SERVOS_QUEUE_NAME # замените на правильное название очереди
        servos_q: Queue = self._queues_dir.get_queue(servos_q_name)

        # отправка сообщения с желаемой скоростью
        event_speed = Event(source=BaseControlSystem.event_source_name,
                      destination=servos_q_name,
                      operation="set_speed", parameters=speed
                    ) # замените на код создания сообщения со скоростью для приводов
                           # подсказка, требуемая операция - set_speed

        # отправка сообщения с желаемым направлением
        event_direction = Event(source=BaseControlSystem.event_source_name,
                      destination=servos_q_name,
                      operation="set_direction", parameters=direction
                    ) # замените на код создания сообщения с направлением для приводов
                               # подсказка, требуемая операция - set_direction

        servos_q.put(event_speed)
        servos_q.put(event_direction)  
        
    def _lock_cargo(self):
        """ заблокировать грузовой отсек """
        cargo_q = self._queues_dir.get_queue(CARGO_BAY_QUEUE_NAME)
        event = Event(source=CONTROL_SYSTEM_QUEUE_NAME,
                    destination=CARGO_BAY_QUEUE_NAME,
                    operation="lock_cargo", parameters= None
                    )
        cargo_q.put(event)

    def _release_cargo(self):
        """ открыть грузовой отсек """
        cargo_q = self._queues_dir.get_queue(CARGO_BAY_QUEUE_NAME)
        event = Event(source=CONTROL_SYSTEM_QUEUE_NAME, 
                    destination=CARGO_BAY_QUEUE_NAME,
                    operation="release_cargo", parameters= None
                    )  
        cargo_q.put(event)      



class CommunicationGateway(BaseCommunicationGateway):
    """CommunicationGateway класс для реализации логики взаимодействия
    с системой планирования заданий

    Работает в отдельном процессе, поэтому создаётся как наследник класса Process
    """
    def _send_mission_to_consumers(self):
        """ метод для отправки сообщения с маршрутным заданием в систему управления """
        
        # имена очередей блоков находятся в файле src/config.py
        # события нужно отправлять в соответствие с диаграммой информационных потоков
        control_q_name = CONTROL_SYSTEM_QUEUE_NAME

        # события передаются в виде экземпляров класса Event, 
        # описание класса находится в файле src/event_types.py
        event = Event(source=BaseCommunicationGateway.event_source_name,
                      destination=control_q_name,
                      operation="set_mission", parameters=self._mission
                      )

        # поиск в каталоге нужной очереди (в данном случае - системы управления)
        control_q: Queue = self._queues_dir.get_queue(control_q_name)
        # отправка события в найденную очередь
        control_q.put(event)    
        


class NavigationSystem(BaseNavigationSystem):
    """ класс навигационного блока """
    def _send_position_to_consumers(self):        
        control_q_name = CONTROL_SYSTEM_QUEUE_NAME # замените на правильное название очереди
        event =  Event(source=BaseControlSystem.event_source_name,
                      destination=control_q_name,
                      operation="position_update", parameters=self._position
                    ) # замените на код создания сообщения с координатами для системы управления 
                     # подсказка, требуемая операция - position_update
        control_q: Queue = self._queues_dir.get_queue(control_q_name)
        control_q.put(event) 


Если у вас настроена и работает СУПА, установите в True значение переменной afcs_present

In [10]:
afcs_present = True

В этом модуле изменим идентификатор машинки:

In [11]:
car_id = "m2" 

### Модуль 2. Отправка маршрутного здания и следование по маршруту

В рамках этого модуля отрабатывается только автономность. 
Никаких киберпрепятствий пока не будет. 

Что нужно сделать по пунктам:
1. Для заданных начальных и конечных точек создать маршрут, содержащий не менее пяти промежуточных путевых точек вручную или с использованием таких инструментов как APMPlanner, MissionPlanner. <br>Содержимое wpl файла нужно вставить в соответствующий кодовый блок блокнота, запустить симуляцию и убедиться, что машинка успешно проходит этот маршрут.
2. Задать скоростные ограничения в виде массива элементов GeoSpecificSpeedLimit
3. Запустить симуляцию


Формат WPL файла описан на этой странице: 

https://mavlink.io/en/file_formats/#mission_plain_text_file

Пример содержимого файла с текстовым описанием маршрута

In [12]:
wpl_file_content =  """QGC WPL 110
0	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.88151910	29.83004810	100.000000	1
1	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.88191210	29.82829930	100.000000	1
2	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.88180980	29.82770920	100.000000	1
3	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.88185830	29.82740880	100.000000	1
4	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.88108840	29.82674360	100.000000	1
5	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.87966170	29.82544540	100.000000	1
6	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.87858490	29.82450130	100.000000	1
7	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.87819720	29.82414720	100.000000	1
8	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.87810030	29.82412580	100.000000	1
9	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.87802490	29.82428670	100.000000	1
10	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.87748110	29.82676510	100.000000	1
11	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.87691580	29.82914690	100.000000	1
12	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.87685120	29.82913610	100.000000	1
13	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.87653890	29.82881430	100.000000	1
14	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.87615120	29.82848170	100.000000	1
15	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.87565310	29.82748660	100.000000	1
16	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.87529030	29.82771860	100.000000	1
17	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.87519270	29.82761260	100.000000	1
18	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.87504200	29.82788090	100.000000	1
19	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.87514430	29.82844950	100.000000	1
20	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.87513350	29.82896450	100.000000	1
21	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.87499350	29.82954380	100.000000	1
22	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.87476740	29.82985500	100.000000	1
23	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.87445500	29.82998370	100.000000	1
24	0	3	16	0.00000000	0.00000000	0.00000000	0.00000000	59.87426660	29.82988720	100.000000	1
"""


Для выполнения задания в блоке ниже  
1. замените имя файла с маршрутом (на свой), вот эту строку:
   ```python
   wpl_file = "module2.wpl"
   ```
2. измените список скоростных ограничений, вот этот массив:
   ```python
   speed_limits = [
    GeoSpecificSpeedLimit(...
    ```
   
Учтите, что вам нужно доставить груз как можно быстрее, но при этом не нарушить правила дорожного движения. 

Ограничения скорости
- населённый пункт - 60 км/ч
- пешеходная зона - 20 км/ч
- магистраль - 110 км/ч

Составьте маршрут доставки так, чтобы машинка доехала за минимальное время. Для определения оптимального маршрута допустимо пользоваться сторонними сервисами (например, Яндекс картами).

In [13]:
# создадим wpl файл и запишем туда маршрутное задание
wpl_file = "module_2.wpl"

**Примечание**:
После создания собственного маршрута удалите перезапись содержимого, чтобы не потерять свои наработки!


In [5]:
# with open(wpl_file, "w") as f:
#     f.write(wpl_file_content)

**Напоминание**: не забудьте изменить скоростные ограничения (speed_limits) при задании маршрутного задания!

In [14]:
from src.wpl_parser import WPLParser

parser = WPLParser(wpl_file)    
points = parser.parse()
print(points)

from src.mission_type import GeoSpecificSpeedLimit
speed_limits = [
    GeoSpecificSpeedLimit(0, 20),
    GeoSpecificSpeedLimit(3, 60),
    GeoSpecificSpeedLimit(6, 20),
    GeoSpecificSpeedLimit(8, 60),
    GeoSpecificSpeedLimit(11, 20),
]

from src.mission_planner import Mission

home = points[0]
mission = Mission(home=home, waypoints=points,speed_limits=speed_limits, armed=True)

[Point(59.8815191, 29.8300481, 0.0), Point(59.8819121, 29.8282993, 0.0), Point(59.8818098, 29.8277092, 0.0), Point(59.8818583, 29.8274088, 0.0), Point(59.8810884, 29.8267436, 0.0), Point(59.8796617, 29.8254454, 0.0), Point(59.8785849, 29.8245013, 0.0), Point(59.8781972, 29.8241472, 0.0), Point(59.8781003, 29.8241258, 0.0), Point(59.8780249, 29.8242867, 0.0), Point(59.8774811, 29.8267651, 0.0), Point(59.8769158, 29.8291469, 0.0), Point(59.8768512, 29.8291361, 0.0), Point(59.8765389, 29.8288143, 0.0), Point(59.8761512, 29.8284817, 0.0), Point(59.8756531, 29.8274866, 0.0), Point(59.8752903, 29.8277186, 0.0), Point(59.8751927, 29.8276126, 0.0), Point(59.875042, 29.8278809, 0.0), Point(59.8751443, 29.8284495, 0.0), Point(59.8751335, 29.8289645, 0.0), Point(59.8749935, 29.8295438, 0.0), Point(59.8747674, 29.829855, 0.0), Point(59.874455, 29.8299837, 0.0), Point(59.8742666, 29.8298872, 0.0)]


Запускаем симуляцию с новым маршрутным заданием

In [15]:
from time import sleep
from geopy import Point as GeoPoint


from src.queues_dir import QueuesDirectory
from src.servos import Servos
from src.sitl import SITL
from src.cargo_bay import CargoBay
from src.mission_planner import MissionPlanner
from src.config import LOG_ERROR, LOG_INFO
from src.mission_planner_mqtt import MissionSender
from src.sitl_mqtt import TelemetrySender
from src.system_wrapper import SystemComponentsContainer


# координата текущего положения машинки
home = GeoPoint(latitude=59.8815191, longitude=29.8300481) 


# каталог очередей для передачи сообщений между блоками
queues_dir = QueuesDirectory() 

if afcs_present:
    mission_sender = MissionSender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)
    telemetry_sender = TelemetrySender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)

mission_planner = MissionPlanner(
    queues_dir, afcs_present=afcs_present, mission=mission)

sitl = SITL(
    queues_dir=queues_dir, position=home,
    car_id=car_id, post_telemetry=afcs_present, log_level=LOG_ERROR)


communication_gateway = CommunicationGateway(
    queues_dir=queues_dir, log_level=LOG_ERROR)
control_system = ControlSystem(queues_dir=queues_dir, log_level=LOG_INFO)

navigation_system = NavigationSystem(
    queues_dir=queues_dir, log_level=LOG_ERROR)

servos = Servos(queues_dir=queues_dir, log_level=LOG_ERROR)
cargo_bay = CargoBay(queues_dir=queues_dir, log_level=LOG_INFO)

# у нас получилось довольно много блоков, используем класс SystemComponentsContainer
# для упрощения рутинной работы с ними - таким образом мы собираем все блоки машинки в одном "кузове"
system_components = SystemComponentsContainer(
    components=[
        mission_sender,
        telemetry_sender,
        sitl,
        mission_planner,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system
    ] if afcs_present else [
        sitl,
        mission_planner,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system
    ])

system_components.start()

# ограничение поездки по времени
# параметр sleep - время в секундах,
# настройте этот параметр так, чтобы ваша машинка завершила маршрут
# в случае превышения времени выполнения ячейки на более чем 10 секунд от заданного, 
# допустимо перезапустить вычислительное ядро и повторно выполнить весь блокнот, штрафные очки за это не начисляются
# при условии, что повторный запуск закончился успешно
sleep(250)

# останавливаем все компоненты
system_components.stop()

# удалим все созданные компоненты
system_components.clean()

[ИНФО][QUEUES] создан каталог очередей
[ИНФО][QUEUES] регистрируем очередь planner.mqtt
[ИНФО][QUEUES] регистрируем очередь sitl.mqtt
[ИНФО][QUEUES] регистрируем очередь planner
[ИНФО][MISSION PLANNER] создана система планирования заданий
[ИНФО][QUEUES] регистрируем очередь sitl
[ИНФО][QUEUES] регистрируем очередь communication
[ИНФО][QUEUES] регистрируем очередь control
[ИНФО][CONTROL] создана система управления
[ИНФО][QUEUES] регистрируем очередь navigation
[ИНФО][QUEUES] регистрируем очередь servos
[ИНФО][QUEUES] регистрируем очередь cargo
[ИНФО][CARGO] создан компонент грузового отсека, отсек заблокирован
[ИНФО][MISSION PLANNER] старт системы планирования заданий
[ИНФО][CARGO] старт блока грузового отсека
[ИНФО][MISSION PLANNER] запрошена новая задача, отправляем получателям[ИНФО][CONTROL] старт системы управления

[ИНФО][MISSION PLANNER] новая задача отправлена в коммуникационный шлюз
[ИНФО][CONTROL] установлена новая задача, начинаем следовать по маршруту, текущее время 15:51:47.

На этом модуль 2 завершён, надеемся, у вас всё получилось! 
На что можно ещё обратить вниманием - можно ли как-то улучшить маршрут для ускорения доставки? (без нарушения ПДД) - например, использовать более скоростные трассы или срезать какие-то участки.

Если идей быстрых улучшений нет, то пришло время перейти к самому интересному модулю и познакомиться с киберпрепятствиями!

Для работы над модулем 3 задания откройте блокнот cyberimmunity--autonomous-car-m3.